<a href="https://colab.research.google.com/github/eltonaguiar/LoanerLab/blob/main/Copy_of_cheapwinners_plus_majorbigfish_grok_update_backup_APIs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import userdata

try:
    alpha_vantage_key = userdata.get('ALPHA_VANTAGE_KEY')
    print("ALPHA_VANTAGE key retrieved successfully:", alpha_vantage_key)
except Exception as e:
    print("Error accessing ALPHA_VANTAGE:", str(e))

# Test other keys
try:
    tiingo_key = userdata.get('TIINGO_KEY')
    print("TIINGO_KEY retrieved successfully:", tiingo_key)
except Exception as e:
    print("Error accessing TIINGO_KEY:", str(e))

ALPHA_VANTAGE key retrieved successfully: SKYZQAMZHNXFBKDL
TIINGO_KEY retrieved successfully: 2247aa4e93338de698597f58f44136f08e17694d


In [1]:
pip install python-dotenv


In [8]:
# Add import for Google Colab secrets
try:
    from google.colab import userdata
except ImportError:
    # Fallback for non-Colab environments
    import os
    userdata = None

# Config
# Load API keys from Colab Secrets if running in Colab, otherwise fallback to environment variables
def get_secret_or_env(secret_name, env_name):
    try:
        if userdata:
            return userdata.get(secret_name)
        return os.getenv(env_name, 'your_key_here')
    except Exception as e:
        logging.error(f"Failed to load {secret_name}: {str(e)}")
        return 'your_key_here'

CONFIG = {
    'ALPHA_VANTAGE_KEY': get_secret_or_env('ALPHA_VANTAGE_KEY', 'ALPHA_VANTAGE'),
    'TIINGO_KEY': get_secret_or_env('TIINGO_KEY', 'TIINGO_KEY'),
    'FINNHUB_KEY': get_secret_or_env('FINNHUB_KEY', 'FINNHUB_KEY'),
    'POLYGON_KEY': get_secret_or_env('POLYGON_KEY', 'POLYGON_KEY'),
    'MARKETSTACK_KEY': get_secret_or_env('MARKETSTACK_KEY', 'MARKETSTACK'),
    'QUANDL_KEY': get_secret_or_env('QUANDL_KEY', 'QUANDL_KEY'),
    'FRED_KEY': get_secret_or_env('FRED_KEY', 'FRED_KEY'),
    'STOCK_API_PROVIDER': 'alpha_vantage',
    'RISK_FREE_RATE': 0.05,
    'MAX_HISTORY_DAYS': 90,
    'STOCK_API_CALLS_PER_MINUTE': 5,
    'TARGET_STOCKS': 15,
    'MAX_PRICE_THRESHOLD': 8.0,
    'PREDICTION_HORIZONS': {
        'hours': 6,
        'days': 3,
        'weeks': 2
    }
}

# Validate that API keys are properly loaded
def validate_api_keys():
    missing_keys = []
    for key_name, key_value in CONFIG.items():
        if 'KEY' in key_name and (key_value is None or key_value == 'your_key_here'):
            missing_keys.append(key_name)
    if missing_keys:
        logging.warning(f"Missing or invalid API keys: {missing_keys}")
        print(f"⚠️ Warning: The following API keys are missing or invalid: {missing_keys}")
        print("Please ensure they are correctly set in Colab Secrets (case-sensitive) or environment variables.")
    else:
        print("✅ All API keys loaded successfully.")

# Rest of your imports
import logging
import time
import argparse
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from typing import Dict, List, Optional
from scipy import stats
import yfinance as yf
import requests
from sklearn.linear_model import LinearRegression

# Rate limiting
def rate_limit_api_calls():
    """Implement rate limiting for API calls"""
    time.sleep(60 / CONFIG['STOCK_API_CALLS_PER_MINUTE'])

# API failure tracker
API_FAIL_TRACKER = {
    'total_failures': 0,
    'symbols_failed': set()
}

# get_us_market_stocks
def get_us_market_stocks() -> List[str]:
    logging.info("Fetching list of US market stocks...")
    fallback_stocks = [
        "RVYL", "OTRK", "IDN", "TUYA", "AIRG", "SNDL", "CPRX", "FCEL", "NAK",
        "CIDM", "GSAT", "SOLO", "IDEX", "GEVO", "ONTX",
        "AAPL", "TSLA", "NVDA", "AMZN", "GOOGL", "MSFT", "META", "AMD", "XOM",
        "JNJ", "JPM", "BLK", "DAL",
        "COUR", "GHG", "HGBL", "HPE", "DTCK", "IQ", "LFST", "NGD", "PTON", "PAYO",
        "EMX", "TARA", "TGB", "MASS", "BLDE", "BZUN", "CRDF", "PLBY", "PRCH",
        "CVRX", "NEOG", "CRVO", "DYN", "VREX", "HPK", "NTIC", "RGNX", "SUNS",
        "LUNG", "ZYME", "RDFN", "FOLD", "IVR", "BOOM", "MAZE", "ALMS", "DEA",
        "NRIX", "CRGY"
    ]
    return list(set(fallback_stocks))

# get_stock_data
def get_stock_data(symbol: str, days: int = 90) -> Optional[pd.DataFrame]:
    logging.info(f"Fetching data for {symbol} (past {days} days)...")
    end_date = datetime.now()
    start_date = end_date - timedelta(days=days)

    api_providers = [
        ('alpha_vantage', CONFIG['ALPHA_VANTAGE_KEY'], lambda: fetch_alpha_vantage(symbol, start_date, end_date)),
        ('tiingo', CONFIG['TIINGO_KEY'], lambda: fetch_tiingo(symbol, start_date, end_date)),
        ('finnhub', CONFIG['FINNHUB_KEY'], lambda: fetch_finnhub(symbol, start_date, end_date)),
        ('polygon', CONFIG['POLYGON_KEY'], lambda: fetch_polygon(symbol, start_date, end_date)),
        ('marketstack', CONFIG['MARKETSTACK_KEY'], lambda: fetch_marketstack(symbol, start_date, end_date)),
        ('quandl', CONFIG['QUANDL_KEY'], lambda: fetch_quandl(symbol, start_date, end_date)),
        ('yahoo', None, lambda: fetch_yahoo(symbol, start_date, end_date))
    ]

    for provider, api_key, fetch_func in api_providers:
        for attempt in range(2):
            try:
                rate_limit_api_calls()
                df = fetch_func()
                if df is not None and not df.empty:
                    logging.info(f"Successfully fetched {symbol} from {provider}")
                    return df
                logging.warning(f"Attempt {attempt + 1} failed for {provider} with {symbol}")
            except Exception as e:
                logging.warning(f"{provider} attempt {attempt + 1} failed for {symbol}: {e}")

        logging.error(f"All attempts for {provider} failed for {symbol}")

    API_FAIL_TRACKER['symbols_failed'].add(symbol)
    API_FAIL_TRACKER['total_failures'] += 1
    logging.error(f"All APIs failed for {symbol}. Total failures: {API_FAIL_TRACKER['total_failures']}")

    if API_FAIL_TRACKER['total_failures'] >= 3 and len(API_FAIL_TRACKER['symbols_failed']) >= 3:
        logging.error("Aborting: 3 failures across 3 symbols reached")
        return None

    return None

# Helper functions for each API
def fetch_alpha_vantage(symbol: str, start_date: datetime, end_date: datetime) -> Optional[pd.DataFrame]:
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&outputsize=full&apikey={CONFIG['ALPHA_VANTAGE_KEY']}"
    response = requests.get(url)
    data = response.json()
    if 'Time Series (Daily)' not in data:
        return None
    df = pd.DataFrame(data['Time Series (Daily)']).T
    df.columns = ['open', 'high', 'low', 'close', 'volume']
    df = df.apply(pd.to_numeric)
    df.index = pd.to_datetime(df.index)
    return df.sort_index().loc[df.index >= start_date.strftime('%Y-%m-%d')]

def fetch_tiingo(symbol: str, start_date: datetime, end_date: datetime) -> Optional[pd.DataFrame]:
    url = f"https://api.tiingo.com/tiingo/daily/{symbol}/prices?startDate={start_date.strftime('%Y-%m-%d')}&endDate={end_date.strftime('%Y-%m-%d')}&token={CONFIG['TIINGO_KEY']}"
    headers = {'Content-Type': 'application/json'}
    response = requests.get(url, headers=headers)
    data = response.json()
    if not data:
        return None
    df = pd.DataFrame(data)
    df = df.rename(columns={'date': 'index', 'adjOpen': 'open', 'adjHigh': 'high', 'adjLow': 'low', 'adjClose': 'close', 'adjVolume': 'volume'})
    df['index'] = pd.to_datetime(df['index'])
    df = df.set_index('index')
    return df[['open', 'high', 'low', 'close', 'volume']]

def fetch_finnhub(symbol: str, start_date: datetime, end_date: datetime) -> Optional[pd.DataFrame]:
    start_timestamp = int(start_date.timestamp())
    end_timestamp = int(end_date.timestamp())
    url = f"https://finnhub.io/api/v1/stock/candle?symbol={symbol}&resolution=D&from={start_timestamp}&to={end_timestamp}&token={CONFIG['FINNHUB_KEY']}"
    response = requests.get(url)
    data = response.json()
    if 's' in data and data['s'] == 'no_data':
        return None
    df = pd.DataFrame({
        'open': data['o'],
        'high': data['h'],
        'low': data['l'],
        'close': data['c'],
        'volume': data['v']
    }, index=pd.to_datetime(data['t'], unit='s'))
    return df

def fetch_polygon(symbol: str, start_date: datetime, end_date: datetime) -> Optional[pd.DataFrame]:
    url = f"https://api.polygon.io/v2/aggs/ticker/{symbol}/range/1/day/{start_date.strftime('%Y-%m-%d')}/{end_date.strftime('%Y-%m-%d')}?apiKey={CONFIG['POLYGON_KEY']}"
    response = requests.get(url)
    data = response.json()
    if 'results' not in data:
        return None
    df = pd.DataFrame(data['results'])
    df = df.rename(columns={'o': 'open', 'h': 'high', 'l': 'low', 'c': 'close', 'v': 'volume', 't': 'index'})
    df['index'] = pd.to_datetime(df['index'], unit='ms')
    df = df.set_index('index')
    return df[['open', 'high', 'low', 'close', 'volume']]

def fetch_marketstack(symbol: str, start_date: datetime, end_date: datetime) -> Optional[pd.DataFrame]:
    url = f"http://api.marketstack.com/v1/eod?access_key={CONFIG['MARKETSTACK_KEY']}&symbols={symbol}&date_from={start_date.strftime('%Y-%m-%d')}&date_to={end_date.strftime('%Y-%m-%d')}"
    response = requests.get(url)
    data = response.json()
    if 'data' not in data:
        return None
    df = pd.DataFrame(data['data'])
    df = df.rename(columns={'open': 'open', 'high': 'high', 'low': 'low', 'close': 'close', 'volume': 'volume', 'date': 'index'})
    df['index'] = pd.to_datetime(df['index'])
    df = df.set_index('index')
    return df[['open', 'high', 'low', 'close', 'volume']]

def fetch_quandl(symbol: str, start_date: datetime, end_date: datetime) -> Optional[pd.DataFrame]:
    url = f"https://www.quandl.com/api/v3/datasets/WIKI/{symbol}.json?start_date={start_date.strftime('%Y-%m-%d')}&end_date={end_date.strftime('%Y-%m-%d')}&api_key={CONFIG['QUANDL_KEY']}"
    response = requests.get(url)
    data = response.json()
    if 'dataset' not in data:
        return None
    df = pd.DataFrame(data['dataset']['data'], columns=data['dataset']['column_names'])
    df = df.rename(columns={'Date': 'index', 'Open': 'open', 'High': 'high', 'Low': 'low', 'Close': 'close', 'Volume': 'volume'})
    df['index'] = pd.to_datetime(df['index'])
    df = df.set_index('index')
    return df[['open', 'high', 'low', 'close', 'volume']]

def fetch_yahoo(symbol: str, start_date: datetime, end_date: datetime) -> Optional[pd.DataFrame]:
    stock = yf.Ticker(symbol)
    df = stock.history(start=start_date.strftime('%Y-%m-%d'), end=end_date.strftime('%Y-%m-%d'))
    df.columns = [col.lower() for col in df.columns]
    return df if not df.empty else None

# calculate_volatility
def calculate_volatility(stock_data: pd.DataFrame) -> float:
    """Annualized volatility based on daily returns"""
    returns = stock_data['close'].pct_change()
    return returns.std() * np.sqrt(252)

# sharpe_ratio
def sharpe_ratio(stock_data: pd.DataFrame, risk_free_rate: float = CONFIG['RISK_FREE_RATE']) -> float:
    returns = stock_data['close'].pct_change()
    avg_return = returns.mean() * 252
    volatility = calculate_volatility(stock_data)
    return (avg_return - risk_free_rate) / volatility if volatility else 0

# max_drawdown
def max_drawdown(stock_data: pd.DataFrame) -> float:
    cumulative = (1 + stock_data['close'].pct_change().fillna(0)).cumprod()
    peak = cumulative.cummax()
    drawdown = (cumulative - peak) / peak
    return drawdown.min()

# predict_stock_movement
def predict_stock_movement(stock_data: pd.DataFrame, horizon: str = 'days') -> Dict:
    X = np.arange(len(stock_data)).reshape(-1, 1)
    y = stock_data['close'].values
    model = LinearRegression()
    model.fit(X, y)
    future_steps = CONFIG['PREDICTION_HORIZONS'].get(horizon, 3)
    future_price = model.predict(np.array([[len(stock_data) + future_steps]]))[0]
    last_price = stock_data['close'].iloc[-1]
    movement_pct = ((future_price - last_price) / last_price) * 100
    return {
        'symbol': stock_data.name if hasattr(stock_data, 'name') else 'Unknown',
        'current_price': last_price,
        'predicted_price': future_price,
        'movement_percentage': movement_pct,
        'direction': 'UP' if movement_pct > 0 else 'DOWN'
    }

# assess_stock_risk
def assess_stock_risk(symbol: str, stock_data: pd.DataFrame) -> Dict:
    volatility = calculate_volatility(stock_data)
    sharpe = sharpe_ratio(stock_data)
    drawdown = max_drawdown(stock_data)
    score = (volatility * 50) + (abs(drawdown) * 30) - (sharpe * 20)
    categories = {
        'Very Low Risk': (0, 10),
        'Low Risk': (10, 25),
        'Moderate Risk': (25, 50),
        'High Risk': (50, 75),
        'Very High Risk': (75, 100)
    }
    risk_category = next((cat for cat, (low, high) in categories.items() if low <= score < high), 'Extreme Risk')
    return {
        'symbol': symbol,
        'volatility': volatility,
        'sharpe_ratio': sharpe,
        'max_drawdown': drawdown,
        'risk_score': score,
        'risk_category': risk_category
    }

# explain_prediction
def explain_prediction(stock):
    explanation = f"\n\nExplanation for {stock['symbol']} Recommendation:\n"
    explanation += f"1. 🧠 High School Level: This stock's price has been going up steadily. Our model thinks it will keep going up, and it's not very risky compared to others.\n"
    explanation += f"2. 👨‍💻 Technical (Coders): Linear regression on {stock['symbol']}'s closing price showed a positive slope. This trend, extrapolated, suggests a future increase.\n"
    explanation += f"3. 📊 Statistical Insight: We regressed closing prices (y) on time (t), observed a significant upward trend.\n"
    explanation += f"   With a Sharpe ratio of {stock['sharpe_ratio']:.2f} and drawdown of {stock['max_drawdown']:.2f}, this asset shows favorable risk-adjusted return.\n"
    explanation += f"   Risk category: {stock['risk_category']}\n"
    explanation += "\nPotential Enhancements:\n"
    explanation += "- Replace regression with XGBoost using volume, RSI, and news sentiment\n"
    explanation += "- Add earnings surprise filter and insider trade flag\n"
    return explanation

# display_intro
def display_intro():
    print("\n====================")
    print("📈 STOCK PREDICTOR OVERVIEW")
    print("====================")
    print("This script analyzes U.S. stocks (primarily under $8, but also top tech picks) using:")
    print("- 📊 Linear Regression to predict short-term price trends")
    print("- ⚖️ Sharpe Ratio, Volatility, Max Drawdown to assess risk")
    print("- 🔎 Filters to select low-risk, high-upside candidates")
    print("Goal: Identify undervalued stocks likely to move up over the next few days.")
    print(f"Data range pulled: Last {CONFIG['MAX_HISTORY_DAYS']} days")
    print("API Priority: Alpha Vantage → Tiingo → Finnhub → Polygon → Marketstack → Quandl → Yahoo Finance\n")

# Update main to validate keys before starting
def main(verbose: bool = False):
    logging.basicConfig(level=logging.INFO)
    print("🔍 Starting stock analysis...")

    # Validate API keys before proceeding
    validate_api_keys()

    display_intro()
    print("📦 Fetching stock list...")

    symbols = get_us_market_stocks()
    print(f"✅ Retrieved {len(symbols)} stock symbols. Beginning data collection and prediction...")

    if not symbols:
        print("\nERROR: No stock symbols were retrieved. Please check your internet connection and API key.")
        return

    print("🔄 Processing stock symbols, please wait...")
    start_time = time.time()
    processed = 0
    total_symbols = len(symbols)
    eligible_stocks = []

    for idx, symbol in enumerate(symbols, 1):
        try:
            processed += 1
            elapsed = time.time() - start_time
            est_total_time = (elapsed / processed) * total_symbols if processed > 0 else 0
            est_remaining = est_total_time - elapsed
            print(f"⏳ [{processed}/{total_symbols}] Processing {symbol} | Est. Time Left: {int(est_remaining)}s")
            print(f"   ↪️ Pulling last {CONFIG['MAX_HISTORY_DAYS']} days of price data...")
            print("   📌 Goal: Predict short-term direction and evaluate risk profile")

            if verbose:
                logging.info(f"Processing stock {processed}/{total_symbols}: {symbol}")

            stock_data = get_stock_data(symbol, days=CONFIG['MAX_HISTORY_DAYS'])
            if stock_data is None or stock_data.empty:
                if API_FAIL_TRACKER['total_failures'] >= 3 and len(API_FAIL_TRACKER['symbols_failed']) >= 3:
                    print(f"\n⚠️ Aborted: Reached 3 failures across {len(API_FAIL_TRACKER['symbols_failed'])} symbols")
                    break
                continue

            current_price = stock_data['close'].iloc[-1]
            if current_price > CONFIG['MAX_PRICE_THRESHOLD']:
                continue

            stock_data.name = symbol
            prediction = predict_stock_movement(stock_data)
            risk = assess_stock_risk(symbol, stock_data)
            stock_info = {**prediction, **risk}
            eligible_stocks.append(stock_info)

            if len(eligible_stocks) >= CONFIG['TARGET_STOCKS']:
                break

        except Exception as e:
            logging.warning(f"Error processing {symbol}: {e}")

    print("✅ Stock processing complete. Ranking results...")

    if eligible_stocks:
        ranked = sorted(eligible_stocks, key=lambda x: (x['risk_score'], -x['movement_percentage']))
        print("\n======================================")
        print(f"Top Low-Price Stock Predictions (under ${CONFIG['MAX_PRICE_THRESHOLD']}):")
        print("======================================")

        for i, stock in enumerate(ranked[:10]):
            print(f"\n{i+1}. {stock['symbol']}:\n   Current Price: ${stock['current_price']:.2f}\n   Predicted Price: ${stock['predicted_price']:.2f}\n   Movement: {stock['direction']} by {abs(stock['movement_percentage']):.2f}%\n   Volatility: {stock['volatility']:.4f}\n   Sharpe Ratio: {stock['sharpe_ratio']:.4f}\n   Max Drawdown: {stock['max_drawdown']:.4f}\n   Risk Category: {stock['risk_category']}")
            print(explain_prediction(stock))
    else:
        print("\nNo stocks matching criteria were found.")
        print("Possible issues:")
        print("1. API rate limits exceeded")
        print("2. Network connection problem")
        print("3. No stocks under the price threshold ($" + str(CONFIG['MAX_PRICE_THRESHOLD']) + ")")
        print("4. API key issues (check your Colab Secrets)")

    if API_FAIL_TRACKER['total_failures'] > 0:
        print(f"\n⚠️ Warning: Encountered {API_FAIL_TRACKER['total_failures']} failures across symbols: {API_FAIL_TRACKER['symbols_failed']}")

    print("✅ Analysis complete. Results above.")

if __name__ == "__main__":
    import sys
    verbose = '--verbose' in sys.argv
    main(verbose=verbose)

🔍 Starting stock analysis...
✅ All API keys loaded successfully.

📈 STOCK PREDICTOR OVERVIEW
This script analyzes U.S. stocks (primarily under $8, but also top tech picks) using:
- 📊 Linear Regression to predict short-term price trends
- ⚖️ Sharpe Ratio, Volatility, Max Drawdown to assess risk
- 🔎 Filters to select low-risk, high-upside candidates
Goal: Identify undervalued stocks likely to move up over the next few days.
Data range pulled: Last 90 days
API Priority: Alpha Vantage → Tiingo → Finnhub → Polygon → Marketstack → Quandl → Yahoo Finance

📦 Fetching stock list...
✅ Retrieved 67 stock symbols. Beginning data collection and prediction...
🔄 Processing stock symbols, please wait...
⏳ [1/67] Processing AMZN | Est. Time Left: 0s
   ↪️ Pulling last 90 days of price data...
   📌 Goal: Predict short-term direction and evaluate risk profile
⏳ [2/67] Processing FOLD | Est. Time Left: 445s
   ↪️ Pulling last 90 days of price data...
   📌 Goal: Predict short-term direction and evaluate ri

ERROR:root:All attempts for alpha_vantage failed for RGNX


⏳ [28/67] Processing DYN | Est. Time Left: 533s
   ↪️ Pulling last 90 days of price data...
   📌 Goal: Predict short-term direction and evaluate risk profile


ERROR:root:All attempts for alpha_vantage failed for DYN


⏳ [29/67] Processing AMD | Est. Time Left: 551s
   ↪️ Pulling last 90 days of price data...
   📌 Goal: Predict short-term direction and evaluate risk profile


...

⏳ [66/67] Processing VREX | Est. Time Left: 31s
   ↪️ Pulling last 90 days of price data...
   📌 Goal: Predict short-term direction and evaluate risk profile


ERROR:root:All attempts for alpha_vantage failed for VREX


⏳ [67/67] Processing BLK | Est. Time Left: 0s
   ↪️ Pulling last 90 days of price data...
   📌 Goal: Predict short-term direction and evaluate risk profile


ERROR:root:All attempts for alpha_vantage failed for BLK


✅ Stock processing complete. Ranking results...

Top Low-Price Stock Predictions (under $8.0):

1. NAK:
   Current Price: $5.10
   Predicted Price: $4.78
   Movement: DOWN by 6.32%
   Volatility: 0.8097
   Sharpe Ratio: 3.7207
   Max Drawdown: -0.2945
   Risk Category: Extreme Risk


Explanation for NAK Recommendation:
1. 🧠 High School Level: This stock's price has been going up steadily. Our model thinks it will keep going up, and it's not very risky compared to others.
2. 👨‍💻 Technical (Coders): Linear regression on NAK's closing price showed a positive slope. This trend, extrapolated, suggests a future increase.
3. 📊 Statistical Insight: We regressed closing prices (y) on time (t), observed a significant upward trend.
   With a Sharpe ratio of 3.72 and drawdown of -0.29, this asset shows favorable risk-adjusted return.
   Risk category: Extreme Risk

Potential Enhancements:
- Replace regression with XGBoost using volume, RSI, and news sentiment
- Add earnings surprise filter and ins